# Preprocessing data

In [12]:
import numpy as np # numpy
from random import randint # data set을 만들기 위한 random library
from sklearn.preprocessing import MinMaxScaler # data normaliza를 위한 scale 라이브러리

## 데이터 예제
- 13 ~ 65세를 위한 어떤 약을 만들기 위해 임상실험 진행
- 2000명의 표본으로 반은 65세 이하를 대상으로 나머지 반은 65세 이상으로 진행
- 95%의 확률로 65세 이상은 부작용 발생
- 95%의 확률로 65세 미만은 부작용 미발생

In [13]:
# X_data : train 데이터, Y_data : label
# 13 ~ 64세는 95%는 부작용 미발생, 5% 발생
# 65세 이상(100세 이하)는 95%는 부작용 발생, 5%는 미발생
# 2000개의 데이터 = 950(13 ~64 : 0) + 50(13 ~ 64 : 1) + 950(65 ~ 100 : 1) + 50(65 ~ 100 : 0)

# Train data 생성, 나이는 13 ~ 64세로 랜덤하게 만들며 1000개의 샘플을 만듭니다.
X_data = np.random.randint(low = 13, high = 64, size = 1000) 
# Test data 생성, 나이는 13 ~ 64세이고 95%는 무작용 미발생, 5%는 발생하도록 만듭니다.
Y_data = np.zeros(950) # 13 ~ 64세 95%는 부작용 미발생
Y_data = np.append(Y_data, np.ones(50)) # 13 ~ 64 95%는 부작용 발생

# Training data 생성, 나이는 65~100세로 랜덤하게 만들며 1000개의 샘플을 만듭니다.
X_data = np.append(X_data, np.random.randint(low = 65, high = 100, size = 1000))
# test data 생성, 나이는 65 ~ 100세이고, 95%는 부작용 발생, 5%는 부작용 미발생하도록 만듭니다.
Y_data = np.append(Y_data, np.ones(950)) # 65 ~ 100세 95%는 부작용 발생
Y_data = np.append(Y_data, np.zeros(50)) # 65 ~ 100세 5%는 부작용 미발생

In [14]:
# Input data scale : 너무 큰 input 값은 weight가 수렴하는 데 오래 걸려
# 학습하는 데 시간이 오래 걸리므로 data scale을 통하여 값을 줄여주어 빠르게 학습하도록 합니다.

# [0, 1] 사이의 값으로 스케일 변화를 위한 변수 생성
scalar = MinMaxScaler(feature_range= (0, 1)) 
# (None, 1) shape으로 [0,1] 범위로 스케일 변경
scaled_X_data = scalar.fit_transform(X_data) 

C:\Users\infoe\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\infoe\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\infoe\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [15]:
print("X_data : ", X_data) # Train data
print("X_data.shape : ", X_data.shape) # Train data shape
print("scaled_X_data : ", scaled_X_data) # scaled train data
print(scaled_X_data.shape) # scaled train data shape
print(Y_data) # Label data
print(Y_data.shape) # label data shape

X_data :  [60 15 29 ... 71 93 83]
X_data.shape :  (2000,)
scaled_X_data :  [0.54651163 0.02325581 0.18604651 ... 0.6744186  0.93023256 0.81395349]
(2000,)
[0. 0. 0. ... 0. 0. 0.]
(2000,)


# Neural Network

In [16]:
import keras 
from keras import backend as K 
from keras.models import Sequential # Sequential : model 선언
from keras.layers import Activation # Activation 사용
from keras.layers.core import Dense # Dense layer 사용
from keras.optimizers import Adam # Adam optimizer 사용
from keras.metrics import categorical_crossentropy # cross entropy 사용

### 1. Sequential을 선언한 후 layer를 쌓습니다.

In [17]:
# Sequential()을 이용하여 model을 선언합니다.
model = Sequential()
# model의 전체 틀은 model.summary()를 이용하여 확인하면 보기 편합니다.

# Dense layer를 추가합니다. 1-dim vector → (None, 16)
# activation으로 ReLU를 사용합니다.
model.add(Dense(16, input_shape=(1,), activation = 'relu'))
# Dense layer를 추가합니다. → (None, 32)
# activation으로 ReLU를 사용합니다.
model.add(Dense(32, activation='relu'))
# Dense layer를 추가합니다. → (None, 2)
# 출력층이므로 activation으로 softmax를 사용합니다.
model.add(Dense(2, activation = 'softmax'))

### 2. 쌓은 layer의 정보를 얻습니다. shape 및 parameter

In [18]:
# 각 layer의 output shape과 Parameter의 수를 요약해서 볼 수 있습니다.
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 16)                32        
_________________________________________________________________
dense_5 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 66        
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________


In [25]:
# model.compile을 이용하여 optimizer와 loss를 선언합니다. 
# optimizer(학습 방법) : Adam 
# loss function(loss 계산 방법) : sparse_categorical_cross_entropy (one-hot 아닐 때는 sparse 써야한다.)
# metrics(performance 판단 기준) : accuracy
model.compile(optimizer = Adam(lr = 0.001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['accuracy'])

In [26]:
# model.fit을 통하여 학습을 합니다.
# input data : scaled_X_data 
# output data : Y_data
# batch size : 10
# epochs : 20
# shuffle (epoch 마다 batch 순서가 shuffle 되는 지 유무) : True (default : True)

model.fit(scaled_X_data, Y_data, batch_size=16, epochs=20, shuffle=True)

Epoch 1/20
2000/2000 [==============================] - 1s 329us/step - loss: 0.2059 - acc: 0.9500
Epoch 2/20
2000/2000 [==============================] - 0s 225us/step - loss: 0.2065 - acc: 0.9485
Epoch 3/20
2000/2000 [==============================] - 0s 191us/step - loss: 0.2069 - acc: 0.9480
Epoch 4/20
2000/2000 [==============================] - 0s 206us/step - loss: 0.2055 - acc: 0.9480
Epoch 5/20
2000/2000 [==============================] - 0s 197us/step - loss: 0.2060 - acc: 0.9500
Epoch 6/20
2000/2000 [==============================] - 0s 203us/step - loss: 0.2047 - acc: 0.9500
Epoch 7/20
2000/2000 [==============================] - 0s 199us/step - loss: 0.2051 - acc: 0.9500
Epoch 8/20
2000/2000 [==============================] - 0s 196us/step - loss: 0.2062 - acc: 0.9490
Epoch 9/20
2000/2000 [==============================] - 0s 201us/step - loss: 0.2037 - acc: 0.9500
Epoch 10/20
2000/2000 [==============================] - 0s 196us/step - loss: 0.2051 - acc: 0.9500
Epoch 11/